# Quick Start

The notebook contains basic usage examples of NNDE. Also, running the whole notebook produces the figures included in the paper. 

## Installation

In [ ]:
! pip install -r requirements.txt
! pip install NNDE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import NNDE
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Sampling Tools

In [ ]:
###### generate samples with marginal density (Exp(0.5),Exp(0.5),Uniform(0,5))

from NNDE import ExponentialDistribution, UniformDistribution, MarginalDistribution

# setup
dim=3
density1 = ExponentialDistribution(lamda=0.5) 
density2 = UniformDistribution(low=0,upper=5) 

density_seq=[]
for i in range(dim-1):
    density_seq = density_seq+[density1]
density_seq=density_seq+[density2]

densitymarginal = MarginalDistribution(density_seq)


# generate 10 samples and return their pdf
samples,samples_pdf = densitymix.generate(10)
samples

In [ ]:
# evaluate pdf at given samples
densitymix.density(samples)

# compare with true pdf
samples_pdf



In [ ]:
###### generate samples with mixture density 0.4*Normal(1.5,0.05)+0.6*Normal(-1.5,0.3)

from NNDE import MultivariateNormalDistribution, MixedDistribution

# setup
dim=2
density1 = MultivariateNormalDistribution(mean = np.zeros(dim)+1.5, cov = np.diag(np.ones(dim)*0.05)) 
density2 = MultivariateNormalDistribution(mean = np.zeros(dim)-1.5, cov = np.diag(np.ones(dim)*0.3)) 
density_seq = [density1, density2]
prob_seq = [0.4, 0.6]
densitymix = MixedDistribution(density_seq, prob_seq)

# generate 10 samples and return their pdf
samples, samples_pdf = densitymix.generate(10)
samples

In [ ]:
# evaluate pdf at given samples
densitymix.density(samples)

# compare with true pdf
samples_pdf



## Adaptive KD Tree

The Adaptive KD Tree is able to select largest k such that $R_k^{\beta}*k<C$ for given $\beta$ and $C$.

In [ ]:
###### normal KDTree query as in sklearn

from NNDE import KDTree as AKDTree
X, =densitymix.generate(100)


tree = AKDTree(X, leaf_size=2)              
dist, ind = tree.query(X[:1], k=3)  

dist
ind

In [ ]:
###### adaptive KDTree query 
    
dist, k = tree.adaptive_query(X[:2], C=0.1, beta=1)  

# dist**beta * k should be close to but smaller than C=0.1 for all entries
dist*k

## NN Density Estimators

In [ ]:
###### using AWNN to estimate density

from NNDE import AWNN

# generate samples
X_train, pdf_X_train =densitymix.generate(1000)
X_test, pdf_X_test =densitymix.generate(1000)

# choose parameter C=0.1
model_AWNN=AWNN(C=.1).fit(X_train)
# output is log scaled
est_AWNN=np.exp(model_AWNN.predict(X_test))
# compute the mean absolute error
np.abs(est_AWNN-pdf_X_test).mean()


In [ ]:
###### using KNN to estimate density with k chosen by cross validation with respect to MISE criterion

from NNDE import KNN
from sklearn.model_selection import GridSearchCV

# generate samples
X_train, pdf_X_train =densitymix.generate(1000)
X_test, pdf_X_test =densitymix.generate(1000)

# select parameter grid
parameters={"k":[int(i*1000) for i in [0.01,0.02,0.05,0.1,0.2,0.5]]}
# use all available cpu, use 10 fold cross validation
cv_model_KNN=GridSearchCV(estimator=KNN(),param_grid=parameters,n_jobs=-1,cv=10)
_=cv_model_KNN.fit(X_train)
model_KNN=cv_model_KNN.best_estimator_
    
# output is log scaled
est_KNN=np.exp(model_KNN.predict(X_test))
# compute the mean absolute error
np.abs(est_KNN-pdf_X_test).mean()


## Visualization

In [ ]:
###### 3d prediction surface using WKNN

from NNDE import contour3d
from sklearn.model_selection import GridSearchCV

# generate samples
X_train, pdf_X_train =densitymix.generate(1000)

model_plot=contour3d(X_train,method="WKNN",k=100)
model_plot.estimation()
model_plot.make_plot()


In [ ]:
###### 2d prediction surface using BKNN

from NNDE import contourf2d
from sklearn.model_selection import GridSearchCV

# generate samples
X_train, pdf_X_train =densitymix.generate(1000)

model_plot=contourf2d(X_train,method="BKNN",C=1)
model_plot.estimation()
model_plot.make_plot()


In [ ]:
###### 1d prediction line plot using TKNN

from NNDE import lineplot
from sklearn.model_selection import GridSearchCV

# generate samples
X_train, pdf_X_train =densitymix.generate(1000)

model_plot=lineplot(X_train,methodseq=["TKNN"],k=100,true_density_obj=densitymix)
model_plot.plot()



In [ ]:
###### plot in the paper 

from NNDE import lineplot
from sklearn.model_selection import GridSearchCV

# generate samples
X_train, pdf_X_train =densitymix.generate(1000)

for method in ["KNN", "WKNN", "AKNN", "BKNN", "TKNN", "AWNN"]:
    model_plot=lineplot(X_train,methodseq=,k=100,true_density_obj=densitymix)
    fig=model_plot.plot()
    fig.savefig("./paper/"+method+".pdf",bbox_inches="tight",dpi=600)

